In [63]:
import numpy as np
import nltk
import pandas as pd 
import string
import re
from spellchecker import SpellChecker
from sklearn.feature_extraction.text import TfidfVectorizer

In [70]:
#nltk.download('words')
infl = getAllInflections
i = infl('begin')
inf = []
for key in i.keys():
    inf_k = i[key]
    inf += list(inf_k)


In [90]:
inf + ['hello']

['began', 'begun', 'beginning', 'begins', 'begin', 'begin', 'hello']

In [74]:
i['RB']

KeyError: 'RB'

In [92]:
def get_list_of_inflections(word):
    """ Function takes a word (as a string) and returns a list of 
        its possible inflections using a lemminflect method.
    """
    
    inflections = getAllInflections(word)
    inf = []
    # getAllInflections is a dictionary, and we want a list of the words.
    for key in inflections.keys():
        inf_k = inflections[key]
        inf += list(inf_k)  # Transform the tuples into lists

    return inf        

In [19]:
#len(words.words())
#dic = set(words.words())
#print(len(dic))
fname = 'pos_train.txt'
with open(fname, 'r') as fh:
    lines = fh.readlines()


In [89]:
for i, line in enumerate(lines):
    if i == 10: break
    else:
        print(line.strip())
        

<user> i dunno justin read my mention or not . only justin and god knows about that , but i hope you will follow me #believe 15
because your logic is so dumb , i won't even crop out your name or your photo . tsk . <url>
" <user> just put casper in a box ! " looved the battle ! #crakkbitch
<user> <user> thanks sir > > don't trip lil mama ... just keep doin ya thang !
visiting my brother tmr is the bestest birthday gift eveerrr ! ! !
<user> yay ! ! #lifecompleted . tweet / facebook me to let me know please
<user> #1dnextalbumtitle : feel for you / rollercoaster of life . song cocept : life , #yolo , becoming famous ? <3 14 #followmeplz ! <3 x15
workin hard or hardly workin rt <user> at hardee's with my future coworker <user>
<user> i saw . i'll be replying in a bit .
this is were i belong


In [91]:
def create_dictionary_with_inflections(dictionary):
    """ Function takes an english dictionary (from nltk) as a list, and adds all possible inflections of words
        using the lemminflect module. It then returns a set of such words.
    """
    new_ldic = []
    for word in dictionary:
        inf = get_list_of_inflections(word)
        
        if not inf:  # If the list of inflections is empty, makes sure to add the word still.
            new_ldic += [word]
        else:
            new_ldic += get_list_of_inflections(word)
    
    # Remove redundancies :
    new_dic = set(new_ldic)
    return new_dic

In [86]:
small_alt_dic = create_dictionary_with_inflections(small_dic)

[]
[]
[]
[]
[]
[]
[]
['aardvarks', 'aardvark']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [93]:
alt_dic = create_dictionary_with_inflections(words.words())

In [94]:
len(alt_dic)

263192

In [287]:
def remove_punctuation(text, excepts='!#<\'', numbers=True):
    """ Removes the punctuation from a text, except the ones in except (a string)."""
    text = text.lower()  # Remove caps
    punct = string.punctuation
    
    for symb in excepts:
        punct = punct.replace(symb, '')
        
    text = "".join([char for char in text if char not in punct])
    text = re.sub('[0-1]+', '', text)
    text = re.sub('[5-9]+', '', text)
    if not numbers :
        raise NotImplementedError
    return text
        

In [173]:
def create_reference_dictionary(filename, learning=True, limit=1000000000000000):
    """ This function creates a dictionary of references :
        a pandas dataframe with 5 columns : the word itself, the position of the word in the file as a tuple, 
        the correction of the words (initially the word itself), the tokens and an alternative correction column.
        learning separates the case where we take the words from a learning file or a testing file (they are written differently).
        """
    dat = {"filewords": [], "position": [], "correction" : [], "tokens": [], "alternative correction": []}
    with open(filename, 'r') as fh:
        lines = fh.readlines()
    for i, line in enumerate(lines):
        if i == limit: break
        else:
            s_line = line.strip()
            if not learning:
                # In this case each line starts by its number followed by a comma
                k = 0
                while s_line[k] != ',':
                    k += 1
                # Remove everything up to the comma (included)
                s_line = s_line[k + 1:]
                
            
            # Remove punctuation except #, !, <, >
            s_line = remove_punctuation(s_line)
            tweet_words = s_line.split()
            for j, w in enumerate(tweet_words):
                dat["filewords"].append(w)
                dat["position"].append((i, j))
                dat["correction"].append(w)
                dat["tokens"].append([])
                dat["alternative correction"].append(w)
    
    return pd.DataFrame(dat)
        
    

In [245]:
twitter_abbreviations = [('ofc', 'of course'), ('lmao', 'laughing my ass off', 'haha'), ('lmfao', 'laughing my fucking ass off', 'haha'),
       ('lyvm', 'love you very much'), ('afk', 'away from keyboard'), 
      ('aight', 'alright'), ('aka', 'also known as'), ('lol', 'laughing out loud', 'haha'), 
      ('aisi', 'as i see it'), ('alcon', 'all concerned'), ('asap', 'as soon as possible'), 
      ('atm', 'at the moment'), ('aweso', 'awesome'), ('b', 'be'), ('bf', 'boyfriend'), ('bae', 'baby'), ('bb', 'baby'),
        ('babe', 'baby'), ('bbl', 'be back later'), ('bc', 'because'), ('cuz', 'because'), ('bday', 'birthday'), 
            ('bff', 'best friend forever'), ('bfn', 'bye for now'), ('bg', 'big grin', 'haha'), ('bih', 'burn in hell'),
        ('bl', 'belly laugh', 'haha'), ('bloke', 'man'), ('bn', 'bad news'), ('bol', 'best of luck'), ('boyf', 'boyfriend'),
        ('brb', 'be right back'), ('bs', 'bullshit'), ('brd', 'bored'), ('btw', 'by the way'),
         ('clab', 'crying like a baby'), ('ciao', 'goodbye'), ('y', 'why'), ('cto', 'check this out'), ('cos', 'because'), 
            ('cmon', 'come on'), ('c', 'see'), ('cu', 'see you'), ('cya', 'see you'), ('dafuq', 'what the fuck'), ('wtf', 'what the fuck'),
            ('dc', 'disconnect'), ('dd', 'dear'), ('derp', 'silly'), ('dgaf', 'do not give a fuck'),
            ('dh', 'dear'), ('dhu', 'dinosaur hugs'), ('diy', 'do it yourself'),('dm', 'direct message'), ('dmed', 'direct message'),
            ('dnt', 'do not'), ('dw', 'do not worry'), ('ez', 'easy'), ('fab', 'fabulous'), ('fam','family'),
                ('fb', 'facebook'), ('ffs', 'for fuck sake'), ('fml', 'fuck my life'), ('ftw', 'for the win'),
                ('fyi', 'for your information'), ('gf', 'girlfriend'), ('gb', 'goodbye'), ('gd', 'good'), ('gl', 'good luck'),
            ('gn', 'goodnight'), ('gnight', 'goodnight'), ('gnite', 'goodnight'), ('gtg', 'got to go'), ('hmu', 'hit me up'),
            ('hw', 'homework'), ('idc', 'i do not care'), ('idk', 'i do not know'), ('idgaf', 'i do not give a fuck'),
            ('ik', 'i know'), ('ikr', 'i know right'), ('ily', 'i love you'), ('imo', 'in my opinion'),
            ('imu', 'i miss you'), ('irl', 'in real life'), ('k', 'okay'),  ('lmk', 'let me know'),
            ('lil', 'little'), ('meh', 'shrug'), ('msg', 'message'), ('ngl', 'not going to lie'),('nvm', 'never mind'),
            ('np', 'no problem'), ('nvr', 'never'), ('omg', 'oh my god'), ('omfg', 'oh my fucking god'),
            ('omw', 'on my way'), ('otp', 'one true pairing'), ('otw', 'off to work'), ('peeps', 'people'),
            ('pls', 'please'), ('plz', 'please'), ('ppl', 'people'),('probs', 'probably'),('prolly', 'probably'),
            ('qt', 'cute'), ('rofl', 'rolling on the floor laughing', 'haha'), ('rip','rest in peace'), ('rn','right now'),
            ('rly', 'really'), ('sd', 'sweet dreams'), ('smh', 'shake my head'), ('srsly', 'seriously'),
            ('sry', 'sorry'), ('sup', 'what is up'), ('hbu', 'how about you'), ('stfu', 'shut the fuck up'),
            ('ty', 'thank you'), ('tyvm', 'thank you very much'), ('thx', 'thank you'), ('tbh', 'to be honest'),
            ('thot', 'that whore over there'), ('totes', 'totally'), ('wbu', 'what about you'), ('wtf','what the fuck'),
            ('x', 'kiss'), ('xd', 'smile', 'haha'), ('<3', 'kiss'), ('pic', 'picture'), ('2', 'to'), ('b4', 'before'),
            ('xoxo', 'kiss')]

In [246]:
twitter_abbreviations[2][1]

'laughing my fucking ass off'

In [247]:
twitter_obvious_mistakes = [('dat', 'that'), ('dis', 'this'), ('wud', 'would'), ('da', 'the'), ('enuf', 'enough'),
                           ('gud', 'good'), ('dunno', 'do not know'), ('im', 'i am'), ('ok', 'okay'), ('r', 'are'), ('u', 'you'),
                           ('luv', 'love'), ('ur', 'your'), ('wat', 'what'), ('wut', 'what'), ('y', 'why'), ('ya', 'you'), ('yea', 'yes'), 
                           ('dont', 'do not'), ('cant', 'cannot'), ('wont', 'will not'), ('aint', 'am not'), ('isnt', 'is not'),
                            ('doesnt', 'does not'), ('hasnt', 'has not'), ('havent', 'have not'), ('id', 'i would'),
                            ('theres', 'there is'), ('thats', 'that is'), ('wheres', 'where is'), ('werent', 'were not')]


In [248]:
contractions = {
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he had",
"he'd've": "he would have",
"he'll": "he shall",
"he'll've": "he shall have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"i'd": "I would",
"i'd've": "I would have",
"i'l": "i will",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"isn't": "is not",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so is",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'al": "you all",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

In [249]:
onomatopees = [('pf', 'sigh'), ('mwah', 'kiss'), ('muah', 'kiss')]

In [250]:
def repeat_exclamation(word):
    res = False
    i = 0
    while i < len(word) - 1:
        if word[i] == '!' and word[i+1] == '!':
            res = True
        i += 1
    return res
    

In [286]:
def resolve_punctuation(data, apostrophe_res):
    """ Function reads a pandas dataframe and removes the remaining punctuation marks.
        namely the apostrophes the exclamation marks and the hashtags. """
    
    ex_indices = data.index[data.filewords.str.contains('!')].tolist()
    hash_indices = data.index[data.filewords.str.contains('#')].tolist()
    ap_indices = data.index[data.filewords.str.contains('\'')].tolist()
    
    for exi in ex_indices:
        # Take the raw word
        w = data.iat[exi, 0]
        tokens = data.iat[exi, 3]
        tokens.append('exclmt')
        if repeat_exclamation(w):
            tokens.append('rrrpp')
        data.iat[exi, 3] = tokens
        w  = "".join([char for char in w if char != '!'])
        data.iat[exi, 2] = w
        
    for hi in hash_indices:
        w = data.iat[hi, 0]
        tokens = data.iat[hi, 3]
        tokens.append('hashtagg')
        data.iat[hi, 3] = tokens
        # Create the word without the "#", and put it in the correction column
        w = w.replace('#', '')
        #print(w)
        #w  = "".join([char for char in w if char != '#'])
        data.iat[hi, 2] = w
        
    for ai in ap_indices:
        w = data.iat[ai, 0]
        if w in apostrophe_res.keys():
            w = apostrophe_res[w]
        else:
            w  = "".join([char for char in w if char != '\''])
        data.iat[ai, 2] = w
        
    #return data         
    

In [252]:
def split_list_of_tuples(l):
    """ Retrieve a list of the first elements in each tuples"""
    el1 = []
    for t in l:
        el1.append(t[0])
    return el1

In [253]:
def resolve_abbreviations(data, abbr):
    """ Function takes a list of abbreviation and their correction (a list of tuples), and checks the dataset to correct these mistakes
    """
    abbs = split_list_of_tuples(abbr)
    for i, abb in enumerate(abbs):
        # Find the indices where there's this abbreviation.
        abb_ind = data.index[data.filewords == abb].tolist()
        
        # Correct the abbreviation and keep a token.
        for abb_err in abb_ind:
            data.iat[abb_err, 2] = abbr[i][1]
            tokens = data.iat[abb_err, 3]
            tokens.append('abbrvt')
            # If there's additional information about the abbreviation (e.g. lol)
            if len(abbr[i]) == 3:
                tokens.append(abbr[i][2])
            data.iat[abb_err, 3] = tokens
    #return data

In [255]:
def resolve_obvious_mistakes(data, om):

    mistakes = split_list_of_tuples(om)
    for i, mis in enumerate(mistakes):
        # Find the indices where there's this mistake.
        mis_ind = data.index[data.filewords == mis].tolist()

        # Correct the abbreviation and keep a token.
        for mis_err in mis_ind:
            data.iat[mis_err, 2] = om[i][1]
            tokens = data.iat[mis_err, 3]
            tokens.append('mspld')
            # If there's additional information about the abbreviation (e.g. lol)
            if len(om[i]) == 3:
                tokens.append(om[i][2])
            data.iat[mis_err, 3] = tokens
    #return data

In [256]:
def resolve_onomatopee(data, instances):
    
    onos = split_list_of_tuples(instances)
    for i, ono in enumerate(onos):
        # Find the indices where there's this mistake.
        ono_ind = data.index[data.filewords == ono].tolist()

        # Correct the abbreviation and keep a token.
        for ono_err in ono_ind:
            data.iat[ono_err, 2] = instances[i][1]
            tokens = data.iat[ono_err, 3]
            tokens.append('ooo')
            # If there's additional information about the abbreviation (e.g. lol)
            if len(instances[i]) == 3:
                tokens.append(instances[i][2])
            data.iat[ono_err, 3] = tokens
    
    #return data

In [257]:
laughs = ['hah', 'heh', 'hih', 'huh', 'hoh', 'jaj']
def resolve_laugh(data, haha):
    """ Function reads the data and replaces any word containing a term in haha by a token that expresses
        laughter. """
    for laugh in haha:
        # Find the laugh indices
        ha_ind = data.index[data.filewords.str.contains(laugh)].tolist()
        
        # Interpret these words as laughter :
        for ha_err in ha_ind:
            data.iat[ha_err, 2] = ""
            tokens = data.iat[ha_err, 3]
            tokens.append('haha')
            data.iat[ha_err, 3] = tokens
            
    #return data

In [258]:
def resolve_correction_into_words(data):
    """ Method takes the dataframe, and the index, and will separate the correction into
        multiple words (if it needs it). It will add rows for the new words and update the position of the words
        in the same line that are not affected by the correction."""
    
    for i, corr in enumerate(data.correction):
        data.iat[i, 2] = corr.split() 
    #return data
    

In [259]:
def turn_corrections_and_tokens_into_list(data):
    """ Method returns a list of the corrections and the tokens."""
    l = []
    for i, corr in enumerate(data.correction):
        l += corr
        l += data.iat[i, 3]  # Add the tokens
    return l

In [260]:
def compute_tf_idf_analysis(df1, df2=None):
    """ Method takes one or two dataframes and computes a tf idf analysis on them
        if there are two dataframes given, it will return three dataframes, (the first with the
        tf-idf analysis on df1, the second with the analysis on df2, the third with the analysis on both)
        """
    # Transform the corrections into a long string :
    list_of_corrections1 = turn_corrections_and_tokens_into_list(df1)
    corr_string1 = [" ".join(c for c in list_of_corrections1)]
    
    # Initialize the tf-idf vectorizer.
    vectorizer = TfidfVectorizer()
    
    matrix1 = vectorizer.fit_transform(corr_string1).todense()
    
    # Transform the results into pandas dataframes
    matrix1 = pd.DataFrame(matrix1, columns=vectorizer.get_feature_names())
    
    top_words1 = matrix1.sum(axis=0).sort_values(ascending=False)
    
    frames = []
    frames.append(top_words1)
    
    if df2 is None:
        return frames
    else:
        # Repeat the same methods as above for the second dataframe
        list_of_corrections2 = turn_corrections_and_tokens_into_list(df2)
        corr_string2 = [" ".join(c for c in list_of_corrections2)]
        
        corrss = [corr_string1[0], corr_string2[0]]
        corr_string12 = [" ".join(c for c in corrss)]
        #corr_string12 = corr_string1 + corr_string2  # For the analysis on both.
        
        matrix2 = vectorizer.fit_transform(corr_string2).todense()
    
        # Transform the results into pandas dataframes
        matrix2 = pd.DataFrame(matrix2, columns=vectorizer.get_feature_names())
        
        matrix12 = vectorizer.fit_transform([corr_string1, corr_string2]).todense()
    
        # Transform the results into pandas dataframes
        matrix12 = pd.DataFrame(matrix12, columns=vectorizer.get_feature_names())
        
        top_words2 = matrix2.sum(axis=0).sort_values(ascending=False)
        top_words12 = matrix12.sum(axis=0).sort_values(ascending=False)
        frames.append(top_words2)
        frames.append(top_words12)
        return frames
        

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Etienne\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [106]:
stopword_eg = nltk.corpus.stopwords.words('english')

In [261]:
def remove_stopwords(correction, stopword):
    """ function reads the correction part of data and adds a new column of words without any stopword.
    """
    
    correction = [word for word in correction if word not in stopword]
    return correction

def remove_stopwords_from_dataframe(data, stopword):
    
    for i, corr in enumerate(data.correction):
        data.iat[i, 2] = remove_stopwords(corr, stopword)
    #return data

In [262]:
ps = nltk.PorterStemmer()
def stem(correction):
    
    correction = [ps.stem(word) for word in correction]
    return correction

wn = nltk.WordNetLemmatizer()

def lemmatize(corr):
    corr = [wn.lemmatize(word) for word in corr]
    return corr

def stem_and_lemmatize_data(data):
    
    for i, corr in enumerate(data.correction):
        stemmed_corr = stem(corr)
        data.iat[i, 2] = lemmatize(corr)
    #return data

In [263]:
def still_in_same_tweet(data, j):
    """ Method checks if the next word is in the same tweet"""
    pos1 = data.iat[j, 1]
    pos2 = data.iat[j+1, 1]
    return pos1[0] == pos2[0]
    
def write_new_test_file_from_df(data, filename):
    
    with open(filename, 'w') as fh:
        i = 0
        while i <len(data.position):
            pos = data.iat[i, 1]
            line_number = pos[0]
            # write the line number at the start of each line.
            fh.write("{},".format(line_number))
            j = i
            while still_in_same_tweet(data, j):
                for word in data.iat[j, 2]:  # write each word in correction.
                    fh.write("{} ".format(word))
                for token in data.iat[j, 3]: # write each token
                    fh.write("{} ".format(token))
                j += 1
            if j >= len(data.filewords):  # Check if we arrived at the end of the file.
                break
            # Still need to write the last word (because we basically check if the next word is in the tweet, and write
            # it if it has a successor in the same tweet)
            
            for word in data.iat[j, 2]:  # write each word in the last correction.
                fh.write("{} ".format(word))
            for token in data.iat[j, 3]: # write each token
                    fh.write("{} ".format(token))
                
            fh.write("\n")  # Write a new line
            i = j + 1

In [276]:
def preprocess(filename, learning=True, orthograph=False):
    """ This method takes a filename of tweets and returns a pandas data frame on which the preprocessing has been done.
        orthograph = False asks whether we want to try and run the spellchecker on the remaining words (it can be very long).
        learning = True asks whether we are in a learning phase or testing phase.
        """
    # Create the pandas dataframe (it strips the line number if we are in the testing phase)
    df = create_reference_dictionary(filename, learning)
    
    
    # Resolve successively the different preprocessing phases on df.
    
    # First remove the abbreviations
    resolve_abbreviations(df, twitter_abbreviations)
    
    # Resolve the twitter obvious mistakes (such as dat for that etc)
    resolve_obvious_mistakes(df, twitter_obvious_mistakes)
    
    # Take care of the onomatopeia
    resolve_onomatopee(df, onomatopees)
    
    # Take care of the laughs
    resolve_laugh(df, laughs)
    
    # Take care of the remaining punctuation
    resolve_punctuation(df, contractions)
    
    # Take care of words with repetitions
    # Find the indices where a repetion occurs
    rep_df = find_repeat_indices(df)
    correct_repeat_word(df, rep_df, onomatopees, twitter_abbreviations)
    
    # Up until now each correction was a string, we change them into a list of words
    resolve_correction_into_words(df)
    
    # We can now remove the stop words from the corrections
    remove_stopwords_from_dataframe(df, stopword_eg)
    
    # Lastly, we can stem and lemmatize the corrections
    stem_and_lemmatize_data(df)
    
    # Try to correct some spelling mistakes
    if orthograph:
        raise NotImplementedError
        
    return df

In [152]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Etienne\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\wordnet.zip.


True

In [269]:
test_file_name = 'test_data.txt' 

In [283]:
test_df = preprocess(test_file_name, False)

followback
unitytour
happy42
phasell
predators
nhl
yougetmajorpointsif
needtoreesthisshit
shouldhavereaditwithaglassofwine
tellmewhy
letdown
justranting
words
creativewriting
have

nice
day
everyone
crying
fornoreason
fornoreason
teamfollowback

starshipsvideo

jamesmaslowtok

jamesmaslowtok

jamesmaslowtok

jamesmaslowtok

jamesmaslowtok

notcool
stanford
baylor
bbwanted
dangit
sadpanda
playoffhockey
2hourstogo
win
unpopularopinion
happensallthetime
calgary
jobs
job
thanksforfollowing
followback
dfamily
4
happybirthdaynathan
happybirthdaynathan
happybirthdaynathan
happybirthdaynathan
happybirthdaynathan
444
littlebrother
waystomakemehappy
bbthemes
nuxstore
haveagrweekend
shulerawards
motown
thoughtsduringschool
fuckembro
2
hooligans
birthday
thingsifindattractive
inournorts
playcherlloydonradio
ripmrsbieber
ripmrsbieber
takemetothebeach
twat
offline
tumblrgirl4life
mybiggestfearis
goosebumps
unlucky
ilovejesus
godisgood
godislove
godismychampion
techi
babbat
moderation
3rddegreeseatbe

thissucks
truelove
toosad

trashyourextuesday
betteroffwithoutyou
highasfuck
stupidasfuck
creying
someonetakeme

sjpuns
depressing
oomf
crying
foodchat



smh
newmember
hml
fok
solar
rt
selfesstemelowered
livinglikeaking
alwaysmakeschuckle
ff
fail
tattoo

amen
simplelife
ripnanny
needyouhere
boyfriendvideo
bieberonthevoice
nofriends
boyfriend
sleepdeprivedmax
homelessforanhour
firstyear
tacler
ugh
thankgod
my
askbarts
faredealexpress
shotforme
np
3inabed
worksmurk
bosh
tgif
bestfriend
rainyafternoon
imserious
birminghammail
tommybrown
meningitis
sadfamilyhasthatonething
hatework
perfection
followfriday
thevoice
slut
cantwait
special
sale
toomuchtohandle
armyfriends
stupidcalifornia
paris
wannagoback
foreveralone

asktin
asktin
asktin
asktin
asktin
bestfriend
sleepyeb
kevinsad
twitterpartytime
chasingthesun
thewantedonthevoice
gradea
bashtube
boyfiend
believe
thisisgonnabeawesome
nevergiveup
riseabovehate
imstillwaiting
oneweekfast
shoutoutto
mustfollow
swaggy
follow
socent
lavishwonder

In [266]:
new_test_file_name = 'test_preprocessed_without_tf_idf_and_spellcheck.txt'

In [284]:
write_new_test_file_from_df(test_df, new_test_file_name)

IndexError: index 138899 is out of bounds for axis 0 with size 138899

In [73]:
["".join(u) for u in words]

['hello', 'world']

In [285]:
'#' == '#'

True

In [232]:
ll = create_reference_dictionary(test_file_name, False)

In [233]:
new_ll = resolve_abbreviations(ll, twitter_abbreviations)

In [234]:
new_ll = resolve_obvious_mistakes(ll, twitter_obvious_mistakes)

In [235]:
new_ll = resolve_onomatopee(ll, onomatopees)

In [236]:
new_ll = resolve_laugh(ll, laughs)

In [237]:
new_ll = resolve_punctuation(ll, contractions)

cannot
madam
do not
I am
you all
I would
do not
nhls


I am
jessicas
that is
that is
I will
lauries
he is
that is
cannot
will not
am not
do not
I am
I am
that is
did not

do not
let us
I am
I am
I will
carlins
I am
have not
cannot
I have
I am
do not
it is
I am
I am
you are
I am
sarahs
jahmais
does not
I am
I am
there is
there is
cannot
do not
you are
is not
cannot
I will
I will
I will
I will
I will


you are
cannot
I am
that is
I am
cannot
you are
how is
where is
boos
who is
I am
liers
do not
do not
it is
I will
it is
I will
did not
he is
remotes
it is
I am
they are
I am
I am
have not
I am
I am
I will
you have
you are
it is
I am
do not
did not
do not
cannot
have not


will not

mens
could not
donts
it is
do not
is not
would not
I have
I am
is not
do not
she is


I am
I am
I will
it is
has not
will not
do not
we are
you will
kartinis
kartinis
I am
could not
I am
do not
is not
I am
do not
I will
does not
I am
you will
it is
you are
we will
she is
I am
I am
are not
what is
cannot
thanks
h

you all
I am
cannot
weathers
I am
where is
it is
I am
do not
do not
we will
friends
do not


there is
I will


you have
that is
how is
cannot
it is
have not
I have
directors
you are
directors
you are
I have
writers

I will
do not
it is
do not
I will
kierkegaards
do not
fridays
do not
are not
cannot
I am
I am
what is
let us
do not
I would
cannot
that would
I will
you will
capn
you have
nites
I would
cannot
I am
ones
he is
it is
I am
gods
who is
I am
it is
it is
do not
you are
I am
I have

that is
I am
I am
could not
do not


it is
it will
is not
I am
there is
it is
I am
I am
does not


juergensmeyers
juergensmeyers
winters
winters
do not
it is
that is
it is
was not
will not
it is
is not
cannot
cannot
I have
that is
I am
we will


you are
that is
I am
it is
should have
that is
cannot
cannot
cannot
drivers
drivers
you are
I am
cannot
I am
it is
friends
did not
will not

that is
is not
I am
I have
do not
what is
I will
cannot
mommas
do not
I have
do not
you will
did not
it is
exapples


me

In [238]:
rep_ll = find_repeat_indices(ll)

In [239]:
new_ll = correct_repeat_word(ll, rep_ll, onomatopees, twitter_abbreviations)

In [240]:
new_ll = resolve_correction_into_words(ll)

In [241]:
new_ll = remove_stopwords_from_dataframe(ll, stopword_eg)

In [242]:
new_ll = stem_and_lemmatize_data(ll)

In [174]:
write_fname = "stemmed_and_lemmatized_pos_train.txt"
write_new_test_file_from_df(ll, write_fname)

KeyboardInterrupt: 

In [100]:
t = compute_tf_idf_analysis(ll)

In [101]:
t[0]

user          0.612182
exclmt        0.396509
you           0.303175
to            0.225644
the           0.197906
                ...   
mufcfamily    0.000007
muff          0.000007
mugelo        0.000007
mugger        0.000007
letchu        0.000007
Length: 46455, dtype: float64

## query = '\'2\''
ll.query('filewords == {}'.format(query))

In [243]:
query = '\'haha\''
df.query("filewords == {}".format(query))
#ap_ind =ll.index[ll.filewords.str.contains('\'')].tolist()

In [244]:
ll.loc[ap_ind]

,filewords,position,correction,tokens,alternative correction
29,can't,"(1, 10)",[can't],[],can't
57,ma'am,"(3, 2)",[ma'am],[],ma'am
100,don't,"(5, 16)",[],[],don't
127,i'm,"(7, 11)",[i'm],[],i'm
160,y'all,"(10, 3)",[y'al],[rrrpp],y'all
...,...,...,...,...,...
138809,that's,"(9992, 1)",[that's],[],that's
138813,can't,"(9992, 5)",[can't],[],can't
138825,i'm,"(9994, 1)",[i'm],[],i'm
138827,you're,"(9994, 3)",[],[],you're


In [208]:
q = test_df.iat[29, 0]
test_df.iat[29, 2] = contractions[q]

In [32]:
ap_idx = ll.filewords.str.contains('\'')
omg_idx = ll.index[ll.filewords == 'omg'].tolist()

llex = ll.loc[49]
llap=ll[ap_idx]

In [33]:
omg_idx

[1717,
 3677,
 3892,
 8966,
 9199,
 11229,
 15056,
 15228,
 28951,
 30438,
 32583,
 32604,
 36830,
 42426,
 42488,
 45050,
 46701,
 49153,
 51714,
 51914,
 58391,
 63133,
 77288,
 82132,
 92042,
 93759,
 94304,
 102494,
 102622,
 103705,
 104778,
 109995,
 110597,
 133444,
 134526,
 135407,
 135659,
 140308,
 141264,
 142579,
 144063,
 146980,
 148028,
 148104,
 148310,
 148862,
 150184,
 154509,
 154798,
 154799,
 154800,
 155631,
 157304,
 158598,
 160278,
 160417,
 160626,
 162998,
 164360,
 166321,
 167443,
 168615,
 170788,
 172561,
 175648,
 180736,
 181469,
 181471,
 184207,
 189621,
 190590,
 196376,
 199203,
 202886,
 204177,
 209616,
 209845,
 212615,
 213969,
 217970,
 219923,
 220829,
 224281,
 233502,
 234029,
 237166,
 238932,
 244248,
 247254,
 247346,
 249300,
 249493,
 251993,
 255209,
 256900,
 258613,
 265313,
 265500,
 267780,
 269401,
 270693,
 273809,
 276696,
 278117,
 278717,
 279835,
 280304,
 282672,
 283392,
 284323,
 288780,
 289969,
 291565,
 297934,
 29793

In [11]:
e = [49, 53]
for i in e:
    ll.iat[i, 2] = "!!"


In [12]:
ll.loc[49]

filewords                      !
position                  (2, 7)
correction                    !!
tokens                         0
alternative correction         !
Name: 49, dtype: object

In [20]:
conc = pd.DataFrame({"filewords":"!!","position": [(0, 0)], "correction" : "!!","tokens": 0,"alternative correction": "!!"})
nllex = pd.concat([llex, conc])

In [21]:
nllex

,filewords,position,correction,tokens,alternative correction
49,!,"(2, 7)",!,0,!
53,!,"(2, 11)",!,0,!
70,!,"(3, 15)",!,0,!
81,!,"(4, 10)",!,0,!
82,!,"(4, 11)",!,0,!
...,...,...,...,...,...
1298194,!,"(99988, 10)",!,0,!
1298213,!,"(99989, 5)",!,0,!
1298335,!,"(99998, 6)",!,0,!
1298341,!,"(99998, 12)",!,0,!


In [19]:
llap

,filewords,position,correction,tokens,alternative correction
31,won't,"(1, 7)",won't,0,won't
61,don't,"(3, 6)",don't,0,don't
124,hardee's,"(7, 8)",hardee's,0,hardee's
133,i'll,"(8, 3)",i'll,0,i'll
269,wasn't,"(20, 3)",wasn't,0,wasn't
...,...,...,...,...,...
1298241,',"(99990, 26)",',0,'
1298243,',"(99990, 28)",',0,'
1298288,what's,"(99995, 3)",what's,0,what's
1298318,i'm,"(99997, 9)",i'm,0,i'm


In [118]:
di = {"col1":['a', 'b', 'c'], "col2": [1, 2, 3]}
di2 = {"letters":['a', 'b', 'c'], "numbers": [1, 2, 3]}
test = {'filewords': ['<user>', 'i', 'dunno', 'justin', 'read', 'my'], 'position': [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5)], 'correction': ['<user>', 'i', 'dunno', 'justin', 'read', 'my'], 'tokens': [0, 0, 0, 0, 0, 0], 'alternative correction': ['<user>', 'i', 'dunno', 'justin', 'read', 'my']}

In [119]:
pd.DataFrame(test)

,filewords,position,correction,tokens,alternative correction
0,<user>,"(0, 0)",<user>,0,<user>
1,i,"(0, 1)",i,0,i
2,dunno,"(0, 2)",dunno,0,dunno
3,justin,"(0, 3)",justin,0,justin
4,read,"(0, 4)",read,0,read
5,my,"(0, 5)",my,0,my


In [34]:
def find_spelling_mistakes(filename, dictionary):
    """ This function reads a file and detects words that do not exist in the english dictionary.
        Dictionary is a list of words e.g take them from nltk.
    """
    # Create a set of words
    file_words =  set(open(filename).read().split())
    file_non_existing_words = file_words.difference(dictionary)
    # The problem with this is that is doesn't take
    return file_words, file_non_existing_words

In [95]:
alt_words, alt_non_words = find_spelling_mistakes(fname, alt_dic)

In [96]:
awords = list(alt_words)
anwords = list(alt_non_words)

True

In [36]:
lwords = list(words)
lnwords = list(non_words)

In [99]:
len(lnwords)

44889

In [100]:
len(anwords)

40705

In [38]:
len(lnwords)
print(lnwords[:100])

['funnyyy', 'capricciosa', 'maho', '#leggo', 'songs', '#cmgrchat', 'leaders', 'cancels', '#modernfamily', 'futures', 'cubans', 'spears', '#jaythan', 'condo', 'gv', 'westie', "#capricorn's", 'cabell', "committee's", 'waylon', '#wpc12', 'freee', '#tulisa', '#needthesummerbody', 'iyou', 'slices', 'brings', '#thatsmyboys', '#schitzogiveaway', '1kings', 'sex-c', 'allows', '#lifestyle', 'gooodnigghttt', '#indiawants1d', 'watsons', 'thereee', 'ba3yn', 'ashrald', 'snacks', 'khen', 'montages', 'solano', 'blujays', "esa's", '#todayskicks', 'nanaman', '20.32', 'riddim', 'penalties', '#notashamed', 'puffs', 'all-access', 'wonderbra', 'threequel', '#pens', '#onlyjokingiam', 'zai', 'donnneee', 'offerings', 'wooaah', 'bakein', '#ss4ina', 'belgian', 'fvck', 'areolas', 'mathai', 'fixes', '#cfceasypickings', 'scrubs', '2minutes', 'blackpool', 'lilbeedoee', '1200', '#delenakiss', "levi's", '#nofilter', 'yilina', 'toes', '#xfactormemories', '#youlikedit', 'clownn', 'ipar', 'audley', 'referring', '#sending

In [101]:
print(anwords[:100])

['funnyyy', 'capricciosa', 'maho', '#leggo', '#cmgrchat', 'cubans', '#modernfamily', '#jaythan', 'condo', 'gv', 'westie', "#capricorn's", 'cabell', "committee's", 'waylon', '#wpc12', 'freee', '#tulisa', '#needthesummerbody', 'iyou', '#thatsmyboys', '#schitzogiveaway', '1kings', 'sex-c', '#indiawants1d', '#lifestyle', 'gooodnigghttt', 'watsons', 'thereee', 'ba3yn', 'ashrald', 'khen', 'solano', 'blujays', "esa's", '#todayskicks', 'nanaman', '20.32', 'riddim', '#notashamed', 'wonderbra', 'all-access', 'threequel', '#pens', '#onlyjokingiam', 'zai', 'donnneee', 'wooaah', 'bakein', '#ss4ina', 'belgian', 'fvck', 'areolas', 'mathai', '#cfceasypickings', '2minutes', 'blackpool', 'lilbeedoee', '1200', '#delenakiss', "levi's", '#nofilter', 'yilina', '#xfactormemories', '#youlikedit', 'clownn', 'ipar', 'audley', '#sendinggoodthots', 'fryday', 'mimis', '#retweetif', 'rmmber', 'ayy', 'jollibee', '#gonnasmashit', 'panabaker', 'snah', '#bmegplanet', 'arguin', '#nighttimerunning', "hayden's", 'monsss',

In [103]:
def remove_hashtags_user_url(word_list):
    """ Removes all hashtag, users and url words from a list of words. """
    pathological_characters = ['#', '<']
    corrected_list = []
    for word in word_list:
        if word[0] in pathological_characters:
            continue
        else:
            corrected_list.append(word)
    return corrected_list

In [104]:
w = list(anwords)
cw = remove_hashtags_user_url(w)

In [105]:
print(cw[:100])

['funnyyy', 'capricciosa', 'maho', 'cubans', 'condo', 'gv', 'westie', 'cabell', "committee's", 'waylon', 'freee', 'iyou', '1kings', 'sex-c', 'gooodnigghttt', 'watsons', 'thereee', 'ba3yn', 'ashrald', 'khen', 'solano', 'blujays', "esa's", 'nanaman', '20.32', 'riddim', 'wonderbra', 'all-access', 'threequel', 'zai', 'donnneee', 'wooaah', 'bakein', 'belgian', 'fvck', 'areolas', 'mathai', '2minutes', 'blackpool', 'lilbeedoee', '1200', "levi's", 'yilina', 'clownn', 'ipar', 'audley', 'fryday', 'mimis', 'rmmber', 'ayy', 'jollibee', 'panabaker', 'snah', 'arguin', "hayden's", 'monsss', 'mayans', 'rivera', 'lmaaaoo', 'chiminology', 'ashlee', 'llanas', 'potatopower', 'paro', 'kbsworldtv', 'dormia', 'pepa', 'renee', 'cadillac', 'acorde', 'chermainea', 'eventho', 'andrewx', 'vcr', '763-548-4658', 'imy', 'shelia', 'bobdim', 'bentley', 'uglyyy', 'eatchother', 'keepp', "l'immenso", 'postitive', 'foof', 'girlsxx', 'selff', 'mena', 'blaaa', 'buuurrnss', 'twitteros', 'angello', 'merlefest', 'barder', 'liy

In [122]:
h1 = 'hello'
h2 = 'helllo'
spell = SpellChecker()
len(spell.unknown([h1])) == 1

False

In [97]:
len(misspelled)

33679

In [34]:
for i, w in enumerate(ll.filewords):
    if i == 10: break
    print(w)

<user>
i
dunno
justin
read
my
mention
or
not
only


In [98]:
for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
    print(len(spell.candidates(word)))

strawberrybanana
1
diplock
4
cheeseburger
1
tyre
8
woooweee
1
jug
16
#skynews
1
#supportgroup
1
#hateonit
1
mom
9
marcello
2
thurs
5
blew
6
#weneedit
1
trainer
4
birthday
1
if
28
eye-opening
1
assumes
2
carole
6
herbert
4
rich
25
unit
5
#lifeofabrunette
1
out
9
north
41
frankie
1
#yoosu
1
shortie
1
alktheer
1
sheron
4
domino
4
snagging
1
island
4
#lovelycouple
1
#socialmix
1
fantastic
3
goole
4
graduation
1
like
3
ellie
5
slurry
2
watching
2
receiving
2
granite
20
pamella
2
stent
3
throng
4
for sure
1
#goldcardmember
1
#xelha
1
hurry
1
brother
1
mere
12
amaze
1
night
5
#getheremay
1
#noitsnotfordrugsoranythingbad
1
indescribable
1
the
151
where
7
hihihiii
1
coca-cola
1
igadgetware
1
nakedly
1
jeeta
1
away
50
three
3
twice
4
epicness
1
sparksnoteing


KeyboardInterrupt: 

In [86]:
t = 'test'
u = 'tess'
count = 0
while count < len(t):
    print(t[count])
    count += 1
u==t

t
e
s
t


False

In [196]:
def repeat_identifier(word, spell):
    """ Function takes a word and will answer whether there are repeated characters in order to correct them
        if they are misspelled.
    """
    count = 0
    # if word == spellchecker.correction(word):
    while count < len(word):
        if count + 1 < len(word):
            if word[count] == word[count + 1] and len(spell.unknown([word])) == 1:
                return True
        count += 1
    return False

In [195]:
def find_repeat_indices(data):
    """ Function that returns a list of indices for each word that has repeating letters (in data)."""
    rep = []
    spell = SpellChecker()
    for i, w in enumerate(data.filewords):
        if repeat_identifier(w, spell):
            rep.append(i)
    return rep

In [73]:
rep_ll

[2,
 20,
 21,
 50,
 52,
 54,
 66,
 80,
 89,
 90,
 99,
 102,
 112,
 124,
 133,
 145,
 147,
 159,
 163,
 171,
 176,
 186,
 198,
 205,
 212,
 218,
 229,
 230,
 250,
 261,
 271,
 279,
 280,
 281,
 294,
 298,
 300,
 301,
 316,
 334,
 337,
 339,
 352,
 357,
 362,
 363,
 368,
 371,
 377,
 385,
 389,
 390,
 402,
 425,
 438,
 448,
 462,
 463,
 469,
 470,
 473,
 475,
 491,
 496,
 523,
 530,
 537,
 551,
 562,
 572,
 579,
 585,
 599,
 655,
 669,
 688,
 699,
 702,
 703,
 707,
 709,
 712,
 717,
 722,
 724,
 725,
 736,
 738,
 752,
 754,
 763,
 765,
 771,
 778,
 780,
 790,
 796,
 801,
 810,
 811,
 812,
 816,
 817,
 822,
 834,
 841,
 844,
 846,
 854,
 861,
 865,
 901,
 933,
 938,
 941,
 942,
 952,
 954,
 958,
 968,
 969,
 972,
 974,
 986,
 1001,
 1021,
 1029,
 1035,
 1040,
 1046,
 1054,
 1061,
 1064,
 1070,
 1079,
 1082,
 1088,
 1093,
 1113,
 1120,
 1175,
 1230,
 1232,
 1241,
 1242,
 1256,
 1261,
 1277,
 1297,
 1300,
 1318,
 1320,
 1326,
 1348,
 1357,
 1365,
 1367,
 1390,
 1395,
 1405,
 1425,
 1437,
 1

In [194]:
def correct_repeat_word(data, indices, onomatopee, abbreviations):
    """ Function will correct the words, making sure to check among abbreviations and 
        onomatopees first before using the spell checker."""
    
    abbs = split_list_of_tuples(abbreviations)
    onos = split_list_of_tuples(onomatopee)
    spell = SpellChecker()
    for ind in indices:
        w = data.iat[ind, 0]
        cand_rep = construct_candidates_for_repeat_word(w)
        found = False
        for candidate in cand_rep:
            if candidate in abbs:
                # Check if the candidate is an abbreviation.
                found = True
                abb_index = abbs.index(candidate)
                data.iat[ind, 2] = abbreviations[abb_index][1]
                
                tokens = data.iat[ind, 3]
                tokens.append('rrrpp')
                tokens.append('abbrvt')
                
                if len(abbreviations[abb_index]) == 3:
                    tokens.append(abbreviations[abb_index][2])
                    
            elif candidate in onos:
                # Check if the canditate is an onomatopee.
                found = True
                ono_index = onos.index(candidate)
                data.iat[ind, 2] = onomatopee[ono_index][1]
                
                tokens = data.iat[ind, 3]
                tokens.append('rrrpp')
                tokens.append('ooo')
                
                if len(abbreviations[ono_index]) == 3:
                    tokens.append(abbreviations[ono_index][2])
            
            elif len(spell.unknown([candidate])) != 1:  # Checks if the spelling is correct.
                found = True
                
                data.iat[ind, 2] = candidate
                
                tokens = data.iat[ind, 3]
                tokens.append('rrrpp')
                
        if not found:
            # Take the candidate with the least amount of repetitions in it.
            data.iat[ind, 2] = cand_rep[-1]
            tokens = data.iat[ind, 3]
            tokens.append('rrrpp')
               
    return data       
                
                
                
    

In [131]:
' ' in 'ha ha ho'


True

In [30]:
def make_list_of_corrections(data):
    ls = []
    for i, corr in enumerate(data.correction):
        if " " in corr:
            for word in corr.split():
                ls.append(word)
        else: ls.append(corr)
            
    return ls

In [31]:
def correct_remainder(data, candidates = 3):
    
    spell = SpellChecker()
    corrections = make_list_of_corrections(data)
    mistakes = spell.unknown(corrections)
    
    for mistake in mistakes:
        if len(spell.candidates(mistake)) <= candidates:
            correct = spell.correction(mistake)
            mis_ind = data.index[data.filewords == mistake].tolist()
            
            for ind in mis_ind:
                data.iat[ind, 2] = correct
        else: continue
            
        
    

In [50]:
ll.loc[rep_ll]

,filewords,position,correction,tokens,alternative correction
50,looved,"(2, 8)",looved,0,looved
90,facebook,"(5, 6)",facebook,0,facebook


In [193]:
def find_repeat_markers(word):
    """ Given a word with repeating characters, this method will compute
        two lists :
        repeat_markers : a list of indices where a repetion will occur
        repeat_numbers : a list of such number of repetitions
        
        Example : the word 'heeloo' will return [1, 4] as repeat markers, and
                  [1, 1] as repeat numbers. 
                  """
    count = 0
    # initialise the indicators of repeatedness in the word
    # the repeat_markers will be a list of the first indices where a character will repeat.
    # The repeat_numbers will denote the amount of repetitions.
    repeat_markers = []
    repeat_numbers = []
    
    # Read the word and identify the repeating characters.
    while count < len(word):
        if count + 1 < len(word):
            if word[count] == word[count + 1]:               
                repeater = count
                while repeater + 1 < len(word) and word[repeater] == word[repeater+1]:
                    repeater += 1
                repeat_numbers.append(repeater - count)
                repeat_markers.append(count)
                count = repeater
        count += 1
    
    return repeat_markers, repeat_numbers

In [192]:
def construct_candidates_for_repeat_word(word):
    """ Method takes a word as a string in input, this word must have at least one repeating character
        otherwise an error will occur.
        This method will return a list of words that have less repeating charcters in them (in decreasing order).
        """
    
    repm, repn = find_repeat_markers(word)
    # We take repn, and return an array of all the possibilities we can have for repeated letters :
    # For instance in heeloo, we have repn = [1, 1] therefore all_possibilities will be 
    # np.array([[0,0], [1, 0], [0, 1], [1, 1]])
    all_possibilities = np.array(np.meshgrid(*[range(x+1) for x in repn])).T.reshape(-1, len(repn))
    
    # We sort this array so that it goes from the least changes to be made, to the more changes to be made:
    # So we sort w.r.t the sum of elements in each row
    
    idx = np.argsort(np.sum(all_possibilities, 1))
    sorted_possibilities = np.take(all_possibilities, idx, axis=0)[::-1]  # In decreasing order
    
    candidates = []
    # Create the list of repeated characters
    repeated_chars = []
    for i in repm:
        repeated_chars.append(word[i])
    
    for ind in range(len(sorted_possibilities)):
        case = sorted_possibilities[ind]
        # Create the instance of repeated characters
        rep_chars_in_candidate = []
        for i, s in enumerate(repeated_chars):
            rep_chars_in_candidate.append(s * (case[i] + 1))
        # Now create the candidate
        candidate = word[:repm[0]]
        for index in range(len(repeated_chars)):
            candidate += rep_chars_in_candidate[index]
            if index + 1 != len(rep_chars_in_candidate):
                candidate += word[repm[index] + repn[index] + 1: repm[index + 1]]  # Take the part of the word inbetween repetitions
            else: continue
        
        candidate += word[repm[-1] + repn[-1] + 1:]
        candidates.append(candidate)
        
    return candidates
    

In [76]:
t1 = 'saaallutt'
t2 = 'dacodaac'
t3 = 'aaaaaaaaaaaaaaaaa'
t4 = 'enoooormeeee'


In [20]:
r1, r2 = find_repeat_candidates(t4)


In [79]:
construct_candidates_for_repeat_word(t4)

[[3 3]
 [3 2]
 [2 3]
 [3 1]
 [2 2]
 [1 3]
 [3 0]
 [2 1]
 [1 2]
 [0 3]
 [2 0]
 [1 1]
 [0 2]
 [1 0]
 [0 1]
 [0 0]]
['oooo', 'eeee']
['oooo', 'eee']
['ooo', 'eeee']
['oooo', 'ee']
['ooo', 'eee']
['oo', 'eeee']
['oooo', 'e']
['ooo', 'ee']
['oo', 'eee']
['o', 'eeee']
['ooo', 'e']
['oo', 'ee']
['o', 'eee']
['oo', 'e']
['o', 'ee']
['o', 'e']


['enoooormeeee',
 'enoooormeee',
 'enooormeeee',
 'enoooormee',
 'enooormeee',
 'enoormeeee',
 'enoooorme',
 'enooormee',
 'enoormeee',
 'enormeeee',
 'enooorme',
 'enoormee',
 'enormeee',
 'enoorme',
 'enormee',
 'enorme']

In [21]:
print(r1)
print(r2)

[2, 8]
[4, 4]


In [72]:
range(0, 1)

range(0, 1)

In [128]:
h = [['t', 'e'], ['t', 'e', 'e']]
hu= []
for it in h:
    hu.append("".join(it))
hu

['te', 'tee']

In [25]:
np.array(np.meshgrid([0, 1, 2], [0, 1, 2], [0, 1, 2])).T.reshape(-1, 3)
shape = (3, 3, 3)
q = np.array(np.meshgrid(*[range(x) for x in shape])).T.reshape(-1, len(shape))

In [28]:
q

array([[0, 0, 0],
       [0, 1, 0],
       [0, 2, 0],
       [1, 0, 0],
       [1, 1, 0],
       [1, 2, 0],
       [2, 0, 0],
       [2, 1, 0],
       [2, 2, 0],
       [0, 0, 1],
       [0, 1, 1],
       [0, 2, 1],
       [1, 0, 1],
       [1, 1, 1],
       [1, 2, 1],
       [2, 0, 1],
       [2, 1, 1],
       [2, 2, 1],
       [0, 0, 2],
       [0, 1, 2],
       [0, 2, 2],
       [1, 0, 2],
       [1, 1, 2],
       [1, 2, 2],
       [2, 0, 2],
       [2, 1, 2],
       [2, 2, 2]])

In [20]:
np.sum(q, 1)
idx = np.argsort(np.sum(q, 1))

In [24]:
np.sum(np.take(q, idx, axis=0), 1)

array([0, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4,
       4, 5, 5, 5, 6])

In [18]:
np.sort(q)

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 2],
       [0, 0, 1],
       [0, 1, 1],
       [0, 1, 2],
       [0, 0, 2],
       [0, 1, 2],
       [0, 2, 2],
       [0, 0, 1],
       [0, 1, 1],
       [0, 1, 2],
       [0, 1, 1],
       [1, 1, 1],
       [1, 1, 2],
       [0, 1, 2],
       [1, 1, 2],
       [1, 2, 2],
       [0, 0, 2],
       [0, 1, 2],
       [0, 2, 2],
       [0, 1, 2],
       [1, 1, 2],
       [1, 2, 2],
       [0, 2, 2],
       [1, 2, 2],
       [2, 2, 2]])

In [9]:
np.meshgrid([0, 1, 2], [0, 1, 2], [0, 1, 2])
#shape = (3, 3, 3)
#np.meshgrid(*[range(x) for x in shape])

[array([[[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2]],
 
        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2]],
 
        [[0, 0, 0],
         [1, 1, 1],
         [2, 2, 2]]]),
 array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[1, 1, 1],
         [1, 1, 1],
         [1, 1, 1]],
 
        [[2, 2, 2],
         [2, 2, 2],
         [2, 2, 2]]]),
 array([[[0, 1, 2],
         [0, 1, 2],
         [0, 1, 2]],
 
        [[0, 1, 2],
         [0, 1, 2],
         [0, 1, 2]],
 
        [[0, 1, 2],
         [0, 1, 2],
         [0, 1, 2]]])]

In [31]:
a = ['a', 'b', 'c']
for i, c in enumerate(a, -1):
    print(i)
    print(c)

-1
a
0
b
1
c


In [51]:
s = 'heeelloo'
b = bytearray('elo', encoding='utf8')
pows=[2, 0, 3]
#np.array(['e', 'l', 'o'])*np.array([3, 2, 1])
u = ['e', 'l', 'o']
new_u = []
for ind, rep in enumerate(u):
    new_u.append(rep *pows[ind])

In [52]:
new_u

['ee', '', 'ooo']

In [55]:
i = [1, 2, 3]
i[len(i)]

IndexError: list index out of range